In [57]:
from transformers import WhisperTokenizer, WhisperFeatureExtractor, WhisperProcessor, WhisperForConditionalGeneration

# Create a vocab json including modulation types and 0~31 modulation indices
vocab = {
    "<|endoftext|>": 0,
    " ": 1,
}
vocab_len = len(vocab)
added_tokens = ["<|8APSK|>", "<|8PSK|>", "<|8QAM|>", "<|16APSK|>", "<|16QAM|>", "<|32APSK|>", "<|32QAM|>", "<|BPSK|>", "<|MSK|>", "<|QPSK|>", "<|unknownmod|>"]
for added_token in added_tokens:
    vocab[added_token] = vocab_len
    vocab_len += 1
vocab_len = len(vocab)
for i in range(32):
    added_tokens.append(str(i))
    vocab[f"<|{i}|>"] = vocab_len
    vocab_len += 1

# Write to vocab.json
import json
with open("vocab.json", "w") as f:
    json.dump(vocab, f)

In [58]:
tokenizer = WhisperTokenizer(vocab_file="vocab.json", merges_file="merges.txt", predict_timestamps=True, additional_special_tokens=added_tokens)
tokenizer("<|8PSK|><|endoftext|> 1 3 30 9 <|endoftext|>")

{'input_ids': [0, 3, 0, 0, 0, 14, 0, 16, 0, 43, 0, 22, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}